*I optimized Delta tables for performance using file compaction and Gold tables were optimized to improve BI query speed and ML feature retrieval performance.*

In [0]:
%sql
OPTIMIZE ecommerce.mental_health.silver_user_features;
OPTIMIZE ecommerce.mental_health.gold_ml_features;
OPTIMIZE ecommerce.mental_health.gold_gender_mental_health_summary;
OPTIMIZE ecommerce.mental_health.gold_behavioral_risk_patterns;
OPTIMIZE ecommerce.mental_health.gold_user_risk_predictions;
OPTIMIZE ecommerce.mental_health.gold_user_risk_predictions_fixed;
OPTIMIZE ecommerce.mental_health.gold_ai_risk_summary;



path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1769529256858, 1769529257616, 8, 0, null, List(0, 0), null, 3, 3, 0, 0, null)"


In [0]:

# 1. Read Silver features table
df = spark.table("ecommerce.mental_health.silver_user_features")


# Prepare clean ML Dataset.
# Directly connected to ML Training.
ml_df = df.select(
    "Age",
    "Gender",
    "Daily_Screen_Time_Hours",
    "Sleep_Duration_Hours",
    "Late_Night_Usage",
    "High_Screen_Time",
    "Sleep_Deprived",
    "LateNight_Sleep_Risk",
    "Passive_Usage_Risk",
    "Social_Comparison_Risk",
    "High_Mental_Health_Risk"   # Target
)

ml_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecommerce.mental_health.gold_ml_features")


In [0]:
# Databricks Job-Safe GOLD Layer Notebook

from pyspark.sql.functions import col, avg, count, round

print("Starting Gold Aggregations...")

# 1. Read Silver features table
df = spark.table("ecommerce.mental_health.silver_user_features")
print("Silver rows:", df.count())

# -------------------------------
# 2. Gender Mental Health Summary
# -------------------------------
gold_gender_mental_health_summary = df.groupBy("Gender").agg(
    count("*").alias("Total_Users"),
    avg("Daily_Screen_Time_Hours").alias("Avg_Screen_Time"),
    avg("Sleep_Duration_Hours").alias("Avg_Sleep"),
    avg("GAD_7_Score").alias("Avg_Anxiety_Score"),
    avg("PHQ_9_Score").alias("Avg_Depression_Score"),
    round(avg("High_Mental_Health_Risk") * 100, 2).alias("Percent_High_Risk")
)

gold_gender_mental_health_summary.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecommerce.mental_health.gold_gender_mental_health_summary")

print("Gender summary created")

# -----------------------------------
# 3. Behavioral Risk Pattern Summary
# -----------------------------------
gold_behavioral_risk_patterns = df.groupBy(
    "High_Screen_Time",
    "Sleep_Deprived",
    "LateNight_Sleep_Risk",
    "Passive_Usage_Risk",
    "Social_Comparison_Risk"
).agg(
    count("*").alias("User_Count"),
    round(avg("High_Mental_Health_Risk") * 100, 2).alias("Risk_Percentage")
)

gold_behavioral_risk_patterns.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("ecommerce.mental_health.gold_behavioral_risk_patterns")

print("Behavior risk summary created")

print("Gold layer completed successfully!")


Starting Gold Aggregations...
Silver rows: 8000
Gender summary created
Behavior risk summary created
Gold layer completed successfully!
